In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import wot

import anndata as ad
# import mnnpy as mp
import scanpy as sc

import sklearn.neighbors as nb

# Data loading

FLE_COORDS_PATH ='data/fle_coords.txt'
FULL_DS_PATH = 'data/ExprMatrix.h5ad'
VAR_DS_PATH = 'data/ExprMatrix.var.genes.h5ad'
CELL_DAYS_PATH = 'data/cell_days.txt'
GENE_SETS_PATH = 'data/gene_sets.gmx'
GENE_SET_SCORES_PATH = 'data/gene_set_scores.csv'
CELL_SETS_PATH = 'data/cell_sets.gmt'

coord_df = pd.read_csv(FLE_COORDS_PATH, index_col='id', sep='\t')
days_df = pd.read_csv(CELL_DAYS_PATH, index_col='id', sep='\t')

adata = wot.io.read_dataset(FULL_DS_PATH, obs=[CELL_DAYS_PATH])

days = adata.obs['day'].unique()[:-1]
indices = [adata.obs['day'].isin([day]) for day in days]
adatalist = [adata[index] for index in indices]

pcadata = np.load('adata_dim_400.npy')
pcadatalist = [pcadata[index] for index in indices]



In [2]:
import utils
from utils import *
from rectifiedflow import *

index_pair = (0, 1)
n_obs_list = [pcadatalist[i].shape[0] for i in range(len(pcadatalist))]
intersection_0_1, unpaired_ref_cell_indices_0_1, unpaired_new_cell_indices_0_1 = NNVerification(index_pair)
pairs_train_0_1, ref_crt_pcadata_0_1, new_crt_pcadata_0_1 = utils.gen_pairs_train(index_pair, n_obs_list, unpaired_ref_cell_indices_0_1, unpaired_new_cell_indices_0_1)
iterations = 10000
batchsize = 2048
input_dim = 400

Time pair: day 0.0, day 0.5
4556
3449
Average intersection ratio: 0.7094773728284611


In [3]:
recFlow = RectifiedFlow(index_pair=index_pair, n_obs_list=n_obs_list, model=MLP(input_dim=input_dim, hidden_num=1024), num_steps=1000)
optimizer = torch.optim.Adam(recFlow.model.parameters(), lr=5e-3)

Model on cuda:0


In [4]:
recFlow, loss_curve = train_rectified_flow(recFlow, optimizer, pairs_train_0_1, batchsize, iterations)

100%|██████████| 10001/10001 [02:26<00:00, 68.49it/s]
